# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-31.36,68,0,2.57,CA,1734502386
1,1,vorgashor,67.5833,63.9500,-24.57,100,97,2.97,RU,1734502387
2,2,papatowai,-46.5619,169.4708,17.12,61,100,3.82,NZ,1734502388
3,3,tiksi,71.6872,128.8694,-23.73,98,85,3.43,RU,1734502389
4,4,namibe,-15.1961,12.1522,21.82,75,100,3.20,AO,1734502391


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_df['coords'] = list(zip(city_data_df['Lat'], city_data_df['Lng']))

# Plotting the map with coordinates
map_plot = city_data_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size=10,
    tiles='OSM',  
    color='City',  
    hover_cols=['City', 'Country', 'Max Temp', 'Humidity']
)

# Display the map
map_plot


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria = ( (city_data_df["Max Temp"] >= 25) &
    (city_data_df["Max Temp"] <= 35) &
    (city_data_df["Humidity"] >= 80)
)

# Apply the criteria
filtered_cities_df = city_data_df[criteria]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Drop the 'coords' column from the filtered DataFrame
filtered_cities_df = filtered_cities_df.drop(columns=['coords'])

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,weno,7.4515,151.8468,28.07,82,97,11.43,FM,1734502415
47,47,mili,6.0815,171.7350,27.83,82,97,8.69,MH,1734502445
66,66,san-pedro,4.7485,-6.6363,25.24,92,51,1.78,CI,1734502468
70,70,richards bay,-28.7830,32.0377,25.83,83,100,0.17,ZA,1734502183
83,83,mumford,5.2618,-0.7590,25.65,86,8,2.08,GH,1734502488
86,86,san antonio de pale,-1.4014,5.6325,26.49,82,53,2.25,GQ,1734502492
90,90,luis correia,-2.8792,-41.6669,26.44,82,62,6.52,BR,1734502496
94,94,afaahiti,-17.7500,-149.2833,25.95,80,94,3.17,PF,1734502501
125,125,enewetak,11.3474,162.3373,27.79,80,76,9.16,MH,1734502538
126,126,noosa heads,-26.3833,153.1500,25.83,84,79,6.56,AU,1734502540


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,weno,FM,7.4515,151.8468,82,
47,mili,MH,6.0815,171.7350,82,
66,san-pedro,CI,4.7485,-6.6363,92,
70,richards bay,ZA,-28.7830,32.0377,83,
83,mumford,GH,5.2618,-0.7590,86,
86,san antonio de pale,GQ,-1.4014,5.6325,82,
90,luis correia,BR,-2.8792,-41.6669,82,
94,afaahiti,PF,-17.7500,-149.2833,80,
125,enewetak,MH,11.3474,162.3373,80,
126,noosa heads,AU,-26.3833,153.1500,84,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
   "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
weno - nearest hotel: High Tide Hotel
mili - nearest hotel: No hotel found
san-pedro - nearest hotel: Hôtel La Cité De San Pédro
richards bay - nearest hotel: SUN1 Richards Bay
mumford - nearest hotel: Blue Diamond Beach Spa Resort
san antonio de pale - nearest hotel: No hotel found
luis correia - nearest hotel: Hotel Amaro
afaahiti - nearest hotel: Omati Lodge
enewetak - nearest hotel: No hotel found
noosa heads - nearest hotel: Costa Nova Apartments
avarua - nearest hotel: Paradise Inn
palikir - national government center - nearest hotel: Mangrove Bay Hotel
piti village - nearest hotel: The Cliff Hotel
maunabo - nearest hotel: No hotel found
baganga - nearest hotel: RGD Inn
touros - nearest hotel: Pousada Atlântico
luanda - nearest hotel: Hotel Forum
twon-brass - nearest hotel: KC Hotel
rio hato - nearest hotel: Playa Blanca Hotel & Resort
axim - nearest hotel: Axim Beach Hotel
saint croix - nearest hotel: The Buccaneer
inarajan village - nearest hotel: No hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
23,weno,FM,7.4515,151.8468,82,High Tide Hotel
47,mili,MH,6.0815,171.7350,82,No hotel found
66,san-pedro,CI,4.7485,-6.6363,92,Hôtel La Cité De San Pédro
70,richards bay,ZA,-28.7830,32.0377,83,SUN1 Richards Bay
83,mumford,GH,5.2618,-0.7590,86,Blue Diamond Beach Spa Resort
86,san antonio de pale,GQ,-1.4014,5.6325,82,No hotel found
90,luis correia,BR,-2.8792,-41.6669,82,Hotel Amaro
94,afaahiti,PF,-17.7500,-149.2833,80,Omati Lodge
125,enewetak,MH,11.3474,162.3373,80,No hotel found
126,noosa heads,AU,-26.3833,153.1500,84,Costa Nova Apartments


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df['coords'] = list(zip(hotel_df['Lat'], hotel_df['Lng']))

# Plotting the map with coordinates
hotel_map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat',
    geo=True,
    size=10,
    tiles='OSM',  
    color='Hotel Name',  
    hover_cols=['City', 'Humidity', 'Hotel Name', 'Country']
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Humidity,Country)